load the pre-processed dataset

In [ ]:
df = pd.read_csv("df.csv")

In [ ]:
import re
import nltk
import gensim
import tiktoken
import pandas as pd
import numpy as np
import seaborn as sns
from umap import UMAP
from hdbscan import HDBSCAN
import matplotlib.pyplot as plt
import gensim.corpora as corpora
from bertopic import BERTopic
from sklearn.cluster import KMeans
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from gensim.models.coherencemodel import CoherenceModel
from sklearn.datasets import fetch_20newsgroups
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize
from sklearn.metrics import silhouette_score
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import OpenAI
import matplotlib.pyplot as plt
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
nlp = spacy.load("en_core_web_sm")
nltk.download('punkt')

In [ ]:
import locale
print(locale.getpreferredencoding())
import nltk
nltk.download('punkt')

UTF-8


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!huggingface-cli login


In [ ]:
seed_words = ["Language","Dataset","Corpus Size","Computational","Machine Translation","Tokenization","Costs"
"Interpretability", "Morphology","Semantic","Memory","Skewed distributions","Biased Distributions","Small size",
              "Generalizability","Bias","Hyperparameter","Hardware","Real world","Robustness", "Noisy", "Time",
              "Annotations", "Evaluation", "Diversity", "Segmentation", "Metrics"]

In [ ]:
example_prompt = """
<|system|>You are a helpful, respectful and honest assistant for labeling topics..</s>
<|user|>
I have a topic that contains the following documents:
- Traditional diets in most cultures were primarily plant-based with a little meat on top, but with the rise of industrial style meat production and factory farming, meat has become a staple food.
- Meat, but especially beef, is the word food in terms of emissions.
- Eating meat doesn't make you a bad person, not eating meat doesn't make you a good one.

The topic is described by the following keywords: 'meat, beef, eat, eating, emissions, steak, food, health, processed, chicken'. And a title 'The Evolution of Diets: From Plant-Based to Meat-Centric'.

Based on the information about the topic above, please create a short label of this topic and title. Make sure you to only return the label and title, nothing more.

<|assistant|> Environmental impacts of eating meat
"""


In [ ]:
main_prompt = """<|system|>You are a helpful, respectful and honest assistant for labeling topics..</s>
<|user|>
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short label and title of this topic. Make sure you to only return the label and title, nothing more..</s>
<|assistant|>"""
prompt =  example_prompt + main_prompt


In [ ]:
# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/zephyr-7B-alpha-GGUF",
    model_file="zephyr-7b-alpha.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=50,
    hf=True
)

In [ ]:
from transformers import AutoTokenizer

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")
# Pipeline
generator = pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=50,
    repetition_penalty=1.1
)

# Text generation with Zephyr
zephyr = TextGeneration(generator, prompt=prompt)
representation_model = {"Zephyr": zephyr}

vectorizer_model = CountVectorizer(stop_words="english")

In [ ]:
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(df['Text'], show_progress_bar=False)

topic_model = BERTopic(representation_model=representation_model,vectorizer_model=vectorizer_model
                        , seed_topic_list = seed_words, verbose=True)

topics, probs = topic_model.fit_transform(df['Text'], embeddings)

umap_embeddings = topic_model.umap_model.transform(embeddings)
indices = [index for index, topic in enumerate(topics) if topic != -1]
X = umap_embeddings[np.array(indices)]
labels = [topic for index, topic in enumerate(topics) if topic != -1]

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(X, labels)

0.5239054

In [ ]:
most_representative_words_llm_mistral_7B = topic_model.get_topic_info()
most_representative_words_llm_mistral_7B

,Topic,Count,Name,Representation,Zephyr,Representative_Docs
0,-1,881,-1_model_models_data_work,"[model, models, data, work, training, language...",[\nLimited limitations of the context of the n...,[The limitations of this paper include the abs...
1,0,268,0_languages_language_translation_data,"[languages, language, translation, data, model...",[\n<|>\n<|>\n<|>\n<|>\n<|>\n<|>\n<|>\n<|>\n<|>...,[Limitation indicates that the transformer mod...
2,1,129,1_bias_social_gender_language,"[bias, social, gender, language, dataset, bias...",[\nLimitations other dialectsociations other t...,[We now discuss limitations of causal-debias.I...
3,2,106,2_dialogue_evaluation_chatgpt_conversations,"[dialogue, evaluation, chatgpt, conversations,...","[\n""Improposed to behavior,>\nTitle: ""The labe...",[Our proposed metric is mainly designed for a ...
4,3,78,3_visual_image_multimodal_images,"[visual, image, multimodal, images, model, sce...",[\nLimitations: Limitations: Limitations: Limi...,[Despite the significant advancements made by ...
5,4,62,4_summarization_summaries_summary_model,"[summarization, summaries, summary, model, lon...","[\nLimited to be limited to beams, but not alw...",[We propose a set prediction network for the e...
6,5,59,5_label_entity_noise_data,"[label, entity, noise, data, annotation, domai...",[\nHandlinguplaces areaspectrumoraclearning ta...,[While interdapt can be used as a strategy to ...
7,6,53,6_reasoning_gpt3_models_explanations,"[reasoning, gpt3, models, explanations, concep...",[\nLabel:assistant|>\nLimited|>\nComputational...,"[In this paper, we provide an overview of the ..."
8,7,53,7_question_qa_questions_answer,"[question, qa, questions, answer, model, entit...",[\nLimitations|>\nFuture directions|>\nLimited...,"[In this paper, we studied paragraph-level qag..."
9,8,47,8_clinical_medical_patients_notes,"[clinical, medical, patients, notes, patient, ...",[\n<|>\n<|>\n<|>\n<|>\n<|>\n<|>\n<|>\n<|>\n<|>...,[This study is based on a single clinical coho...


keybert

In [ ]:
df['Representative_Docs'] = df['Representative_Docs'].apply(lambda x: ''.join(map(str, x)))

In [ ]:
from keybert import KeyBERT

kw_model = KeyBERT()
doc_embeddings, word_embeddings = kw_model.extract_embeddings(df["Representative_Docs"])

In [ ]:
keywords = kw_model.extract_keywords(df["Representative_Docs"], doc_embeddings=doc_embeddings, word_embeddings=word_embeddings)
keywords

[[('annotating', 0.3629),
  ('annotations', 0.3525),
  ('prompting', 0.3507),
  ('language', 0.3489),
  ('annotators', 0.3403)],
 [('translations', 0.5158),
  ('multilingual', 0.4878),
  ('lingual', 0.4323),
  ('translation', 0.4008),
  ('languages', 0.3777)],
 [('biases', 0.5127),
  ('bias', 0.5117),
  ('debiasing', 0.4876),
  ('causaldebias', 0.4246),
  ('debiased', 0.4014)],
 [('dialogue', 0.4834),
  ('evaluation', 0.4512),
  ('chatbots', 0.4455),
  ('conversational', 0.4341),
  ('chatbot', 0.4245)],
 [('parsing', 0.3909),
  ('parsers', 0.3448),
  ('captions', 0.3357),
  ('parser', 0.3302),
  ('visual', 0.3207)],
 [('dialogue', 0.3993),
  ('summarization', 0.3801),
  ('dialog', 0.3376),
  ('graphbased', 0.3002),
  ('decoder', 0.2846)],
 [('annotation', 0.479),
  ('labeling', 0.4334),
  ('datasets', 0.3895),
  ('labeled', 0.3825),
  ('dataset', 0.3723)],
 [('deductive', 0.4692),
  ('reasoning', 0.4572),
  ('inference', 0.3876),
  ('annotations', 0.3474),
  ('structured', 0.3447)],
 [

In [ ]:
words_only = [[word for word, _ in sublist] for sublist in keywords]
print(words_only)
# convert list to string
df['Representative_Docs'] = df['Representative_Docs'].apply(lambda x: ''.join(x))
# Tokenize the strings to create a list of lists
list_of_tokens = df['Representative_Docs'].apply(lambda x: x.split()).tolist()

print(list_of_tokens)
word2id = Dictionary( list_of_tokens )

# Coherence model
cm = CoherenceModel(topics= words_only,
                    texts=list_of_tokens,
                    coherence='c_v',
                    dictionary=word2id)

coherence_per_topic = cm.get_coherence_per_topic()
print(coherence_per_topic)

import statistics
statistics.mean(coherence_per_topic)

[['annotating', 'annotations', 'prompting', 'language', 'annotators'], ['translations', 'multilingual', 'lingual', 'translation', 'languages'], ['biases', 'bias', 'debiasing', 'causaldebias', 'debiased'], ['dialogue', 'evaluation', 'chatbots', 'conversational', 'chatbot'], ['parsing', 'parsers', 'captions', 'parser', 'visual'], ['dialogue', 'summarization', 'dialog', 'graphbased', 'decoder'], ['annotation', 'labeling', 'datasets', 'labeled', 'dataset'], ['deductive', 'reasoning', 'inference', 'annotations', 'structured'], ['qa', 'sentences', 'qag', 'retrievals', 'languages'], ['outcomes', 'immunotherapy', 'prognostic', 'clinical', 'prediction'], ['adversarial', 'votetrans', 'attacks', 'overfitting', 'maliciously'], ['rnns', 'dspert', 'encoder', 'transformer', 'encoderdecoder'], ['retrieval', 'knowledge', 'knowledgeintensive', 'knowledgeable', 'conceptnet'], ['bert', 'sentences', 'lexical', 'embeddings', 'contextual'], ['annotations', 'triggers', 'events', 'event', 'trigger'], ['pruning

0.5514967970921535

In [ ]:
words_only = []
for i in range(len(df['Representation'])):
  words_only.append(df['Representation'][i])

In [ ]:
df['Representative_Docs'] = df['Representative_Docs'].apply(lambda x: ''.join(x))
# Tokenize the strings to create a list of lists
list_of_tokens = df['Representative_Docs'].apply(lambda x: x.split()).tolist()

print(list_of_tokens)
word2id = Dictionary( list_of_tokens )

# Coherence model
cm = CoherenceModel(topics= words_only,
                    texts=list_of_tokens,
                    coherence='c_v',
                    dictionary=word2id)

coherence_per_topic = cm.get_coherence_per_topic()
print(coherence_per_topic)

import statistics
statistics.mean(coherence_per_topic)

[["['The", 'limitations', 'of', 'this', 'paper', 'include', 'the', 'absence', 'of', 'experiments', 'on', 'large', 'language', 'models.Previous', 'studies', 'have', 'shown', 'that', 'using', 'high-capacity', 'pre-trained', 'language', 'models', 'can', 'significantly', 'improve', 'the', 'accuracy', 'of', 'answers', 'but', 'also', 'entails', 'an', 'increase', 'in', 'computational', 'overhead.Due', 'to', 'limitations', 'of', 'computational', 'resources,', 'this', 'paper', 'employs', 'a', 'low-capacity', 't5', 'model', 'for', 'experiments.Our', 'experiments', 'have', 'suggested', 'that', 'the', 'proposed', 'iterative', 'prompting', 'method', 'that', 'works', 'with', 'the', 'low-capacity', 'model', 'can', 'achieve', 'comparable', 'results', 'with', 'baseline', 'methods', 'equipping', 'with', 'large', 'models.recent', 'research', 'on', 'large', 'language', 'models', 'has', 'shown', 'that', 'they', 'can', 'learn', 'from', 'few', 'examples', 'and', 'reason', 'well.We', 'believe', 'that', 'it', 

0.5142795224138114